# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.48.0


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config()
experiment_name = 'hyperdrive-experiment'

experiment=Experiment(ws, experiment_name)

# creating compute cluster

amlcompute_cluster_name = "experiment-cluster2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [3]:
#get data: load from workspace
found = False
key = "kaggle-heart-failure-prediction"
description_text = "Heart Failure Prediction dataset from kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://mlstrg225794.blob.core.windows.net/azureml-blobstore-bfad7455-0114-4658-93ad-3eb28d482c62/UI/2023-02-13_103739_UTC/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

For the classification task I'm using a simple logistic regression. It's a good model to start with. The hyperparameter C is an inverse of regulation strength and max_iter is the maximum numbers of iterations to converge. Smaller values of C cause stronger regularization. The Bandit policy prevents that an algorithm is taking too many iterations to every posiible combination of hyperparameters without producing better results. It is based on factor/slack amount of the most successful job.
The Random sampling supports discrete (max_iter) as well as continuous (C) hyperparameters. Random sampling can be used as an initial search with an later refinement for better results. It also supports early termination, so it is a low-budget solution in contrast to Grid sampling or Bayesian sampling.
In addition, I chose accuracy as primary metric because it is a good meassure for a classification and 10 as a total number of run, so the experiment does not take too long.

In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.1, delay_evaluation = 3, evaluation_interval = 1)


if "training" not in os.listdir():
    os.mkdir("./training")
    
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( {
    "C": uniform(0.2, 2),
    "max_iter": choice(10,50,100,150,200)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory = '.',
            script = 'train_lr.py',
            compute_target = compute_target,
            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=10)

In [10]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_414ed57a-1fbf-45be-a06f-62b35876b998
Web View: https://ml.azure.com/runs/HD_414ed57a-1fbf-45be-a06f-62b35876b998?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-226049/workspaces/quick-starts-ws-226049&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-14T20:26:29.735003][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space
[2023-02-14T20:26:30.5902385Z][SCHEDULER][INFO]Scheduling job, id='HD_414ed57a-1fbf-45be-a06f-62b35876b998_1' 
[2023-02-14T20:26:30.6400609Z][SCHEDULER][INFO]Scheduling job, id='HD_414ed57a-1fbf-45be-a06f-62b35876b998_2' 
[2023-02-14T20:26:30.7039056Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_414ed57a-1fbf-45be-a06f-62b35876b998_0' 
[2023-02-14T20:26:30.7970580Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_414ed57a-1fbf-45be-a06f-62b35876b998_1' 
[2023-02-14T20:26:30.8449628Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_414ed57a-

{'runId': 'HD_414ed57a-1fbf-45be-a06f-62b35876b998',
 'target': 'experiment-cluster2',
 'status': 'Completed',
 'startTimeUtc': '2023-02-14T20:26:29.251814Z',
 'endTimeUtc': '2023-02-14T20:28:34.392593Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd5744121-d6b9-4d4c-aa31-2336bbd79d45',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.8',
  'best_child_run_id': 'HD_414ed57a-1fbf-45be-a06f-62b35876b998_0',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_414ed57a-1fbf-45be-a06f-62b35876b998_0'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(best_run)
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n C:',parameter_values[1])
print('\n max_iter:',parameter_values[3])

RunDetails(best_run).show()

Run(Experiment: hyperdrive-experiment,
Id: HD_e8265d5b-8726-4b1f-8cf2-033747d3aee3_1,
Type: azureml.scriptrun,
Status: Completed)
Best Run Id:  HD_e8265d5b-8726-4b1f-8cf2-033747d3aee3_1

 Accuracy: 0.8

 C: 2.5568773701545626

 max_iter: 120


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [31]:
#TODO: Save the best model
from sklearn.linear_model import LogisticRegression
import numpy as np
import joblib
from sklearn.model_selection import train_test_split

#load dataset from starter_files
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')

# Split data into train and test sets.
x = df.loc[:, df.columns != 'DEATH_EVENT']
y = df['DEATH_EVENT']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

model = LogisticRegression(C=float(parameter_values[1]), max_iter=int(parameter_values[3])).fit(x_train, y_train)

joblib.dump(value=model, filename="best-model-hyperdrive.pkl")

['best-model-hyperdrive.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [32]:
#register model
from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="best-model-hyperdrive.pkl", model_name="best-model-hyperdrive")

Registering model best-model-hyperdrive


In [43]:
#Create inference config and deploy model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

#load registered model
model = Model(ws, 'best-model-hyperdrive')

#load yaml file
env = Environment.from_conda_specification("sklearn-env", "conda_dependencies.yml")

#create inference config
inference_config = InferenceConfig(environment=env,
                                   source_directory='.', 
                                   entry_script='./scoring_file_v_2_0_0_hyperdrive.py')

#create deployment config/ deployment as webservice with enabled insights
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    enable_app_insights=True,
    auth_enabled=True) 

#deploy best model 
endpoint = Model.deploy(ws, "best-model-hyperdrive-endpoint", [model], inference_config, deployment_config, overwrite=True)
endpoint.wait_for_deployment(show_output = True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-14 22:10:10+00:00 Creating Container Registry if not exists.
2023-02-14 22:10:10+00:00 Registering the environment.
2023-02-14 22:10:12+00:00 Use the existing image.
2023-02-14 22:10:12+00:00 Generating deployment configuration.
2023-02-14 22:10:12+00:00 Submitting deployment to compute..
2023-02-14 22:10:16+00:00 Checking the status of deployment best-model-hyperdrive-endpoint.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 5cb58e94-3bec-4a44-a5d2-75cc441f5670
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: ",
  "details": []
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 5cb58e94-3bec-4a44-a5d2-75cc441f5670
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: ",
  "details": []
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: 5cb58e94-3bec-4a44-a5d2-75cc441f5670\nMore information can be found using '.get_logs()'\nError:\n{\n  \"code\": \"AciDeploymentFailed\",\n  \"statusCode\": 400,\n  \"message\": \"Aci Deployment failed with exception: \",\n  \"details\": []\n}"
    }
}

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import json
import requests

#scoring uri and key
scoring_uri = ""
key = ""

data = {
    "Inputs":{
        "data": [
            {
                "age": 65,
                "anaemia": 1,
                "creatinine_phosphokinase": 148,
                "diabetes": 1,
                "ejection_fraction": 20,
                "high_blood_pressure": 0,
                "platelets": 162000,
                "serum_creatinine": 1.9,
                "serum_sodium": 136,
                "sex": 1,
                "smoking": 1,
                "time": 30
            }
        ]
        
    },
    "GlobalParameters":{
        "method": "predict"
    }
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
from azureml.core.webservice import Webservice
# load existing web service and print logs
service = Webservice(name='best-model-hyperdrive-endpoint', workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

In [ ]:
#delete webservice
delete_webservice(service)

#delete compute clusters
compute_target.delete()
compute_target.wait_for_completion(show_output=True)
print('compute cluster deleted')

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

